# 综合案例：金融市场数据可视化

通常，当我们提起股票时，首先想到的是价格走势图，展示特定股票的价格如何随时间变化。然而，还有许多其他重要的图表或统计数据可以提高对股票行为的理解，例如它的趋势或动量，一定程度上这也可以帮助我们预测未来的价格。 
 
在本案例中，我们将探索SPY交易所交易基金的历史。

## 导入包

本案例主要利用 pandas 和 ployly 库进行可视化处理。plotly 是一个用于创建交互性数据可视化的Python包，这意味着用户可以在图表上进行缩放、平移、悬停以查看数据，或者通过点击图例来切换数据系列的可见性。这使得数据分析更加直观和动态。它支持多种图表类型，包括散点图、线图、条形图、饼图、热力图、地图、3D图等。

In [44]:
# 读取数据和数据处理的包
import urllib.request
import os
import pandas as pd

# 关于 plotly 画图的包
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

## 读取数据

In [45]:
download_url_prefix = "https://py-101.github.io/python-data-science/datasets/stock/ETFs"
folder_path = os.path.join(os.getcwd(), "../data/stock/ETFs")
download_url = f"{download_url_prefix}/spy.us.txt"

file_name = download_url.split("/")[-1]
file_path = os.path.join(folder_path, file_name)

if not os.path.exists(folder_path):
    # 创建文件夹
    os.makedirs(folder_path)
    print(f"文件夹不存在，已创建。")

    urllib.request.urlretrieve(download_url, file_path)
    print("数据已下载。")
else:
    print(f"文件夹已存在，无需操作。")

文件夹已存在，无需操作。


In [46]:
df = pd.read_table(file_path, delimiter=',')
df.head()

,Date,Open,High,Low,Close,Volume,OpenInt
0,2005-02-25,104.77,106.00,104.68,105.79,70221808,0
1,2005-02-28,105.55,105.68,104.56,105.08,79695344,0
2,2005-03-01,105.22,105.87,105.22,105.62,54607412,0
3,2005-03-02,105.21,106.22,105.10,105.57,73733090,0
4,2005-03-03,105.99,106.20,105.15,105.61,71286823,0


In [47]:
print("Spy DataFrame shape is {}".format(df.shape))
print("OpenInt contains {}".format(set(df['OpenInt'])))

Spy DataFrame shape is (3201, 7)
OpenInt contains {0}


SPY基金的历史价格数据框架由3201行组成，每一行有7列，分别是：日期、开盘价/最高价/最低价/收盘价、成交量计数和持仓数量。OpenInt列只有0个值，所以本案例中将忽略它，专注于其余的信息。

## 可视化展示
### 修改图像默认输出格式（使得展示更美观但非必需）

In [48]:
# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

def plot_scatter(x, y, title):
    fig = go.Figure(go.Scatter(x=x, y=y, name=title))
    fig.update_layout(title_text=title)
    fig.show()

### OHLC图

OHLC图表显示了股票的开盘价($\textbf{O}pen$)、最高价($\textbf{H}igh$)、最低价($\textbf{L}ow$)和收盘价($\textbf{C}lose$)，这可以展示某一天的价格是如何变化的，也可以体会到股票的动量或波动性。线条的顶端代表最高值和最低值，而水平线段代表开盘值和收盘值。在收盘价高于（低于）开盘价的示例点被称为增长（减少）。默认情况下，增长的部分以绿色绘制，而减少的部分以红色绘制。

In [49]:
fig = go.Figure([go.Ohlc(x=df.Date,
                         open=df.Open,
                         high=df.High,
                         low=df.Low,
                         close=df.Close)]) #Figure 创建图表对象
fig.update(layout_xaxis_rangeslider_visible=True) #图表中的 x 轴范围滑块，允许用户在图表中选择特定时间段或数据范围
fig.show()

### 成交量（Volume）
成交量是一个非常基本的指标，它显示了在一定时间段内（例如每日）交易的股票数量，即买入和卖出的数量。尽管它非常简单，但通常被忽视。成交量非常重要，因为它基本上代表了股票交易的活跃程度。更高的成交量数值表示更多人对交易某只股票感兴趣。

我们作出成交量的柱状图，代码如下：

其中 marker_color 设置柱状图的颜色为红色，因为数据过多，大图看是白色的空隙，利用 plotly 交互放大即可看到红色的柱状图。

In [50]:
fig = go.Figure(go.Bar(x=df.Date, y=df.Volume, name='Volume', marker_color='red')) 
fig.show()

### 移动平均线（Moving Averages）
移动平均线（MA）
有助于通过过滤短期价格波动来平滑图表上的股价。我们在一段定义的时间内计算移动平均值，例如最近的9、50或200天。常用的两种平均线包括：

- 简单移动平均线（SMA）- 是在最近的N天（如50、100或200天）内计算的简单平均值，用.rolling()实现。

$$x_{t+1}=\frac{x_t+x_{t-1}+...+x_{t-n+1}}{n}$$

- 指数移动平均线（EMA）- 是一种平均值，其中更大的权重被赋予最近的价格，用.ewm()实现。
$$x_{t+1}=\alpha x_{t}+\alpha(1-\alpha)x_{t-1}+...$$

移动平均线及其交叉点（请参见[黄金交叉](https://www.investopedia.com/terms/g/goldencross.asp)和[死亡交叉](https://www.investopedia.com/terms/d/deathcross.asp)）通常被用作交易信号。

#### .rolling() 函数

例如，平滑三期，则从第三期开始有对应值，且应作为第四期的预测值，因此常常使用shift()函数对rolling后的结果向前移动一个时间单位，使得与相应时间的Close收盘价对齐。

In [53]:
df['Close_rolling']=df['Close'].rolling(3).mean()
df['Close_shift']=df['Close_rolling'].shift()
df[['Close','Close_rolling','Close_shift']]

,Close,Close_rolling,Close_shift
0,105.79,NaN,NaN
1,105.08,NaN,NaN
2,105.62,105.496667,NaN
3,105.57,105.423333,105.496667
4,105.61,105.600000,105.423333
...,...,...,...
3196,258.85,258.296667,257.843333
3197,258.67,258.656667,258.296667
3198,259.11,258.876667,258.656667
3199,258.17,258.650000,258.876667


In [51]:
df['EMA_9'] = df['Close'].ewm(9).mean().shift() #9日的指数移动平均线,.shift() 用于将这些平均线向前移动一个时间单位，以便与收盘价对齐。
df['SMA_50'] = df['Close'].rolling(50).mean().shift() #50日的简单移动平均线
df['SMA_100'] = df['Close'].rolling(100).mean().shift() #100日的简单移动平均线 
df['SMA_200'] = df['Close'].rolling(200).mean().shift() #200日的简单移动平均线

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date, y=df.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_50, name='SMA 50'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_100, name='SMA 100'))
fig.add_trace(go.Scatter(x=df.Date, y=df.SMA_200, name='SMA 200'))
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close', line_color='dimgray', opacity=0.3))
fig.show()

### 相对强度指数 (RSI)
相对强度指数（Relative Strength Index）指示了最近价格变动的幅度。它可以显示一只股票是超买还是超卖。通常情况下，RSI值在70及以上表明一只股票正变得超买/过高估，而在30及以下的数值则可能表示它被超卖。

$RSI\in [0,100]$，具体计算步骤：



1. 计算每个交易日的价格变动（一般是收盘价）：Price Change = 当日收盘价 - 前一日收盘价。

2. 将正的价格变动（Price Change）和负的价格变动分别累加，以计算 n 天内的平均增长日（Average Gain）和平均损失日（Average Loss）。

3. 计算相对强度（RS）：RS = 平均增长日 / 平均损失日。

4. RSI = 100 - (100 / (1 + RS))


In [33]:
def RSI(df, n=14): #RSI的计算周期为14天，但可以根据需要进行调整，以适应不同的时间框架和交易策略
    close = df['Close']
    delta = close.diff()
    delta = delta[1:]
    pricesUp = delta.copy()
    pricesDown = delta.copy()
    pricesUp[pricesUp < 0] = 0
    pricesDown[pricesDown > 0] = 0
    rollUp = pricesUp.rolling(n).mean()
    rollDown = pricesDown.abs().rolling(n).mean()
    rs = rollUp / rollDown
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

num_days = 365
df['RSI'] = RSI(df).fillna(0)
fig = go.Figure(go.Scatter(x=df['Date'].tail(num_days), y=df['RSI'].tail(num_days))) #.tail 用于截取最近365天的值
fig.show()

### 移动平均收敛散度（Moving Average Convergence Divergence）

移动平均收敛散度（[MACD](https://www.investopedia.com/terms/m/macd.asp#toc-learning-from-macd)）是一种指标，显示了两个指数移动平均线（EMA）之间的关系，即12日和26日的EMA。

$$MACD = 12Period EMA - 26Period EMA$$

信号线是MACD线的九周期EMA。

- MACD最适合在日线周期中使用，当MACD线从上方穿过信号线时，用于买入；从下方跌破信号线时，用于卖出。

- MACD可以帮助判断一种证券是否处于超买或超卖状态，提醒交易员价格走势的力度，并警告潜在的价格逆转。

- MACD还可以提醒投资者存在多空分歧（例如，当价格的新高没有MACD的新高来确认，反之亦然），这可能表明潜在的失败和逆转。在信号线交叉之后，建议等待三到四天以确认它不是虚假动作。

In [35]:
EMA_12 = pd.Series(df['Close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['Close'].ewm(span=26, min_periods=26).mean())
MACD = pd.Series(EMA_12 - EMA_26)
MACD_signal = pd.Series(MACD.ewm(span=9, min_periods=9).mean())

fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df.Date, y=df.Close, name='Close'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_12, name='EMA 12'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=EMA_26, name='EMA 26'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=MACD, name='MACD'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.Date, y=MACD_signal, name='Signal line'), row=2, col=1)
fig.add_trace(go.Bar(x=df.Date, y=MACD-MACD_signal, name=' Momentum'), row=2, col=1)

fig.show()

### 随机指标（[Stochastic](https://www.investopedia.com/terms/s/stochasticoscillator.asp#toc-what-is-a-stochastic-oscillator)）
该指标类似于RSI，范围在0到100之间。传统上，超过80被认为是超买范围，而读数低于20被认为是超卖。然而，这些并不总是预示着即将发生逆转；非常强劲的趋势可以在超买或超卖条件下保持很长时间。相反，交易员应该关注随机振荡器的变化，以获取有关未来趋势变化的线索。

随机指标图表通常包括两条线：一条反映每个交易会话的振荡器实际值（慢随机指标），另一条反映它的三日简单移动平均值（快随机指标）。因为价格被认为遵循动量，这两条线的交汇被认为是逆转信号的迹象。



C为最近的收盘价，$L_{14}$ 和 $H_{14}$ 分别代表过去14天中交易的最低价和最高价，慢随机指标$\%K$的计算方式为：
$$\%K=(\frac{C-L_{14}}{H_{14}-L_{14}}\times 100)$$

快随机指标$\%D$的计算方式为：

$$\%D=SMA_3(\%K)$$




In [27]:
def stochastic(df, k, d):
    df = df.copy()
    low_min  = df['Low'].rolling(window=k).min()
    high_max = df['High'].rolling(window=k).max()
    df['stoch_k'] = 100 * (df['Close'] - low_min)/(high_max - low_min)
    df['stoch_d'] = df['stoch_k'].rolling(window=d).mean()
    return df

stochs = stochastic(df, k=14, d=3)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.Date.tail(365), y=stochs.stoch_k.tail(365), name='K stochastic'))
fig.add_trace(go.Scatter(x=df.Date.tail(365), y=stochs.stoch_d.tail(365), name='D stochastic'))
fig.show()